A short snippet showing how to instatiate a TF Saver object that will only load weights for a specific subset of variables in the current graph. 

In [ ]:
import numpy as np
import sys
import tensorflow as tf
import tensorflow.contrib.slim as slim

sys.path.append('../')

import vgg

In [ ]:
# In case you want to rerun this block again, resetting the graph means you don't 
# have to restart the kernel
tf.reset_default_graph()

# Input dimensions
height = 224
width = 224
channels = 3

# The input node to the graph
# These values are what is required by vgg19 for height, width, channels
input_var_initial_value = np.ones([1, height, width, channels])
input_var = tf.Variable(input_var_initial_value, dtype=tf.float32, name='input_var')

# Load the vgg model
with slim.arg_scope(vgg.vgg_arg_scope()):
    logits, end_points = vgg.vgg_19(input_var, num_classes=1000, is_training=False)
    
# Choose a layer to extract features from
predictions = end_points['vgg_19/conv2/conv2_2']

In [ ]:
# Prepare to restore the vgg19 nodes
# Skip trying to restore the input variable since it's new
all_variables = tf.get_collection_ref(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list=all_variables[1:])

In [ ]:
# Set up initializer node
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer(),
                  name='initialize_all')

In [ ]:
# Choose which input data type you want to experiment with, set to 0 or 1
# 0: Run init op before restoring weights
# 1: Run init op after restoring weights
init_order = 0

In [ ]:
# Restore weights of vgg_10 model
checkpoint_path = '../vgg_19.ckpt'
with tf.Session() as sess:
    
    if init_order == 0:
        # This is the correct order
        sess.run(init_op)
        saver.restore(sess, checkpoint_path)
    else: 
        # This is the wrong order
        saver.restore(sess, checkpoint_path)
        sess.run(init_op)
    
    output = predictions.eval()
    
    print(output)